# OpenAI Function Calling

In [ ]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai.api_key = os.environ["OPENAI_API_KEY"]

In [ ]:
import json

# Example hard coded function to return the same weather (API in prod)
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

In [ ]:
# OpenAI Function definition
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "unit": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"]
                },
            },
            "required": ["location"],
        },
    },
]

In [ ]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston in Celcisu?",
    }
]

In [ ]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
)
response = model.invoke(messages, functions=functions)

In [ ]:
json.loads(response.additional_kwargs["function_call"]["arguments"])

In [ ]:
print("Content:", response.content)
print("Additional kwargs:", response.additional_kwargs["function_call"])
args = json.loads(response.additional_kwargs["function_call"]["arguments"])
json.loads(get_current_weather(**args))

- **Message that's not related to the function**

In [ ]:
messages = [
    {
        "role": "user",
        "content": "Hi!",
    }
]
response = model.invoke(messages, functions=functions)

In [ ]:
print("Content:", response.content)
print("Additional kwargs:", response.additional_kwargs)
# args = json.loads(response.additional_kwargs["function_call"]["arguments"])
# json.loads(get_current_weather(**args))

- **Control function call**

In [ ]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston?",
    }
]
response = model.invoke(messages, functions=functions, function_call="none")

print("Content:", response.content)
print("Additional kwargs:", response.additional_kwargs)

In [ ]:
messages = [
    {
        "role": "user",
        "content": "Hi!",
    }
]
response = model.invoke(messages, functions=functions, function_call={"name": "get_current_weather"})

print("Content:", response.content)
print("Additional kwargs:", response.additional_kwargs)

- **Respond after executing the function**

In [ ]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston?",
    }
]
response = model.invoke(messages, functions=functions, function_call={"name": "get_current_weather"})
messages.append(response)

args = json.loads(response.additional_kwargs["function_call"]["arguments"])
weather = get_current_weather(**args)

messages.append(
    {
        "role": "function",
        "name": "get_current_weather",
        "content": weather,
    }
)

response = model.invoke(messages)
response.content